In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras.layers.core import Dense, Dropout
from keras.layers import SimpleRNN
from tensorflow.keras.optimizers import SGD, Adagrad, RMSprop, Adam, Adadelta, Adamax,Nadam
from keras.callbacks import Callback
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from keras.models import Sequential
import timeit
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import Callback, CSVLogger

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
start = timeit.default_timer()
dataframe = pd.read_csv('/content/drive/MyDrive/Research/Deep Learning/LaminatedPlate_RZT_Q9.csv')
dataset = dataframe.values
data = dataset[:,0:55]
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

In [ ]:
start = timeit.default_timer()
nb_set = 50000
nb_test=int(nb_set*0.7)

x_train = data[:nb_test,0:50]
x_test  = data[nb_test:,0:50]

y_train = data[:nb_test,50:55]
y_test  = data[nb_test:,50:55]
print('y_train',y_train)
print(x_train)
#reshpae input data
x_train = x_train.reshape(x_train.shape[0], 1,x_train.shape[1])
x_test = x_test.reshape(x_test.shape[0],1, x_test.shape[1])

input_node  = x_train.shape[2]
output_node = y_train.shape[1]

y_train [[0.42717531 0.58751401 0.54305166 0.44421383 0.45133866]
 [0.40574368 0.51058079 0.67147243 0.46764448 0.37195867]
 [0.35747964 0.64572483 0.67587772 0.34620174 0.35082198]
 ...
 [0.54693528 0.40029007 0.50981177 0.56353489 0.53316111]
 [0.34393485 0.64565891 0.64864504 0.3467091  0.37627994]
 [0.52593228 0.49350649 0.50226939 0.53406208 0.49689995]]
[[0.35280948 0.40013556 0.59792331 ... 0.74769434 0.83185088 0.39443185]
 [0.09663441 0.26536996 0.79194769 ... 0.10097671 0.13580377 0.85830722]
 [0.0186391  0.72832428 0.81665907 ... 0.96961532 0.36463791 0.66442763]
 ...
 [0.8856154  0.88672956 0.14925721 ... 0.57972021 0.22206172 0.55248628]
 [0.2020078  0.6686834  0.99347771 ... 0.5312825  0.0580405  0.81064293]
 [0.85542062 0.30115448 0.54896943 ... 0.06574737 0.43060641 0.22933751]]


In [ ]:
model = Sequential()
model.add(SimpleRNN(units=300, return_sequences=True,input_shape=(1,input_node), activation="relu"))
model.add(SimpleRNN(units=200,return_sequences=True, activation="relu"))
model.add(SimpleRNN(units=200,return_sequences=True, activation="relu"))
model.add(SimpleRNN(units=100, activation="relu"))
model.add(Dense(50, activation="relu"))
model.add(Dense(output_node))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_14 (SimpleRNN)   (None, 1, 300)            105300    
                                                                 
 simple_rnn_15 (SimpleRNN)   (None, 1, 200)            100200    
                                                                 
 simple_rnn_16 (SimpleRNN)   (None, 1, 200)            80200     
                                                                 
 simple_rnn_17 (SimpleRNN)   (None, 100)               30100     
                                                                 
 dense_4 (Dense)             (None, 50)                5050      
                                                                 
 dense_5 (Dense)             (None, 5)                 255       
                                                                 
Total params: 321,105
Trainable params: 321,105
Non-tr

In [ ]:
#optimizers
adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
rms = RMSprop(learning_rate = 0.01, rho=0.9, epsilon=1e-08)
ada = Adagrad(learning_rate=0.2, epsilon=1e-08)
sgd = SGD(learning_rate=0.1, momentum=0.1)
adadel=Adadelta(learning_rate=1.0, rho=0.95, epsilon=1e-08)
adamax = Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
nadam = Nadam(learning_rate=0.002, beta_1=0.9, beta_2=0.999)

In [ ]:
json_string = model.to_json()
open('/content/drive/MyDrive/Research/Deep Learning/RNN/model.json', 'w').write(json_string)

# model.compile(loss='mean_squared_error', optimizer='adadel', metrics=['accuracy'])
model.compile(loss='mse', optimizer=adamax, metrics=['accuracy'])

In [ ]:
csv_logger = CSVLogger('/content/drive/MyDrive/Research/Deep Learning/RNN/training_history.csv')
history = model.fit(x_train, y_train, epochs = 1000, validation_data=(x_test, y_test),batch_size=256, verbose=1,callbacks=[csv_logger])
scores = model.evaluate(x_test, y_test, verbose=0)
model.save_weights('/content/drive/MyDrive/Research/Deep Learning/RNN/target_weight.h5', overwrite=True)
print(model.evaluate(x_train, y_train))


In [ ]:
pred_train = model.predict(x_train)
pred_test = model.predict(x_test)
mse_train = mean_squared_error(pred_train, y_train)
mse_test  = mean_squared_error(pred_test, y_test)
mae_train = mean_absolute_error(pred_train, y_train)
mae_test = mean_absolute_error(pred_test,  y_test)
r2_train = r2_score(y_train,pred_train)
r2_test = r2_score(y_test,pred_test)

In [ ]:
stop = timeit.default_timer()
with open("/content/drive/MyDrive/Research/Deep Learning/RNN/MSE_Computingtime.txt", "w") as text_file:
    print("Computing time: "+str(stop - start), file=text_file)
    print('MSE for train: ' +str(mse_train),file=text_file)
    print('MSE for test: '  + str(mse_test), file=text_file)
    print('mean absolute error for train: '+str(mae_train),file=text_file)
    print('mean absolute error for test: ' + str(mae_test), file=text_file)
    print('r2 score for train:' +str(r2_train),file=text_file)
    print('r2 score for test:' +str(r2_test),file=text_file)
    print('scores: ' +str(scores),file=text_file)
    print("%s:%.2f%%" % (model.metrics_names[1], scores[1] * 100),file=text_file)

In [ ]:
print('Computing time:',stop - start)
print('mean square error for train and test',mse_train, mse_test)
print('mean absolute error for train and test',mae_train, mae_test)
print('r2 score for train and test', r2_train,r2_test)
print('scores ',scores)
print("%s:%.2f%%"%(model.metrics_names[1],scores[1]*100))

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

In [ ]:
# model.summary()
config = model.get_config()
model.get_weights()
print('config',config)
# print(ascii(model)